In [1]:
from premethod import *
from pegasus_fine_tune import *
from generate_pegasus import *
from combine import *
from rank import rank
from rank_training import rank_train
from p_at_1 import p_at_k
from keybart_finetune import *
from keybart_generate import get_pred_Keybart
from bart_finetune import fine_tune_bart
from bart_generate import get_pred_bart
### 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/admin02/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2022-10-10 16:57:30,054 - sentence_transformers.cross_encoder.CrossEncoder - INFO - Use pytorch device: cuda
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/admin02/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
datadir = ['../dataset/EUR-Lex/','../dataset/Wiki500K/']
k_fold = [0,1,2,3,4]
tasks = ['test','train','valid']
models = {'pega':1,'bart':0,'kb':0}
gener = "generate_result/"

In [3]:
# pegasus
if models['pega']:
        Pegasus_fine_tune(datadir[0],tasks[1]+'_finetune.json',tasks[0]+'_finetune.json',"pegasus_save","pegasus_check")
        for i in range(2):
            get_pred_Pegasus_fast(datadir[1],gener+tasks[i]+"_pred.txt",tasks[i]+"_finetune.json","pegasus_save")
            get_combine_list(datadir[1],gener+tasks[i]+"_pred.txt","all_stemlabels.txt",tasks[i]+"_combine_labels.txt")
        rank_train(datadir[1],tasks[1]+"_texts.txt",tasks[1]+"_combine_labels.txt",tasks[1]+"_labels_stem.txt","cr_en")
        rank(datadir[1],tasks[0]+"_texts.txt",tasks[0]+"_combine_labels.txt","cr_en",tasks[0]+"_ranked_labels.txt")
        res = p_at_k(datadir[1],tasks[0]+"_labels_stem.txt",tasks[0]+"_ranked_labels.txt",datadir[1]+"res_pega.txt")


train_dir:../dataset/Wiki500K/train_finetune.json
valid_dir:../dataset/Wiki500K/test_finetune.json
model_svae: ../dataset/Wiki500K/pegasus_save
model_check: ../dataset/Wiki500K/pegasus_check
cuda


2022-10-10 16:57:34,028 - datasets.builder - WARNING - Using custom data configuration default-0be3e47723b3df5d


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
if models['bart']:
        fine_tune_bart(datadir[0],tasks[1]+'_finetune.json',tasks[0]+'_finetune.json','bart_save','bart_check')
        for i in range(2):
            get_pred_bart(datadir[0],gener+tasks[i]+"_pred_ba.txt",tasks[i]+"_finetune.json","bart_save")
            bart_clean(datadir[0]+gener+tasks[i]+"_pred_ba.txt",datadir[0]+gener+tasks[i]+"_pred_ba_c.txt")
            get_combine_list(datadir[0],gener+tasks[i]+"_pred_ba_c.txt","all_stemlabels.txt",gener+tasks[i]+"_combine_labels_ba.txt")
        rank_train(datadir[0],tasks[1]+"_texts.txt",gener+tasks[1]+"_combine_labels_ba.txt",tasks[1]+"_labels_stem.txt","cr_en_ba")
        rank(datadir[0],tasks[0]+"_texts.txt",gener+tasks[0]+"_combine_labels_ba.txt","cr_en_ba",gener+tasks[0]+"_ranked_labels_ba.txt")
        res = p_at_k(datadir[0],tasks[0]+"_labels_stem.txt",gener+tasks[0]+"_ranked_labels_ba.txt",datadir[0]+"ba_res.txt")

In [ ]:
if models['kb']:    
        kb_fine_tune(datadir[0],"kb_save","kb_check")
        fine_tune_keybart(datadir[0],tasks[1]+'_finetune.json',tasks[0]+'_finetune.json','keybart_save','keybart_test')
        for i in range(2):
            get_pred_Keybart(datadir[0],gener+tasks[i]+"_pred_kb.txt",tasks[i]+"_finetune.json","keybart_save")
            bart_clean(datadir[0]+gener+tasks[i]+"_pred_kb.txt",datadir[0]+gener+tasks[i]+"_pred_kb_c.txt")
            get_combine_list(datadir[0],gener+tasks[i]+"_pred_kb_c.txt","all_labels_sterm.txt",gener+tasks[i]+"_combine_labels_kb.txt")

##### K则验证

In [ ]:
for j in k_fold:
    k_dir = "/K_fold/"+"K_"+str(j)+"/"
    Pegasus_fine_tune(datadir[0]+k_dir,tasks[1]+'.json',tasks[0]+'.json',"pegasus_save","pegasus_check")
    for i in range(2):
        get_pred_Pegasus(datadir[0]+k_dir,tasks[i]+"_pred.txt",tasks[i]+".json","pegasus_save")
        get_combine_list(datadir[0]+k_dir,tasks[i]+"_pred.txt","all_stemlabels.txt",tasks[i]+"_combine_labels.txt")
    rank_train(datadir[0]+k_dir,tasks[1]+"_texts.txt",tasks[1]+"_combine_labels.txt",tasks[1]+"_labels_stem.txt","cr_en_"+str(j))
    rank(datadir[0]+k_dir,tasks[0]+"_texts.txt",tasks[0]+"_combine_labels.txt","cr_en_"+str(j),tasks[0]+"_ranked_labels.txt")
    res = p_at_k(datadir[0]+k_dir,tasks[0]+"_labels_stem.txt",tasks[0]+"_ranked_labels.txt",datadir[0]+k_dir+"res.txt")